## Local Inference on GPU
Model page: https://huggingface.co/intfloat/multilingual-e5-base

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/intfloat/multilingual-e5-base)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [1]:
!python --version

Python 3.10.16


In [13]:
import os
os.environ['TORCH_BLAS_PREFER_HIPBLASLT'] = '0'
os.environ['PYTORCH_TUNABLEOP_ENABLED'] = '0'

# Затем импортируйте torch и другие библиотеки
import torch
from sentence_transformers import SentenceTransformer


In [14]:
import torch

print("="*70)
print("🔍 ПРОВЕРКА GPU")
print("="*70)

print(f"PyTorch версия: {torch.__version__}")
print(f"ROCm версия: {torch.version.hip}")
print(f"Доступных GPU: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        props = torch.cuda.get_device_properties(i)
        print(f"\nGPU {i}:")
        print(f"  Имя: {props.name}")
        print(f"  VRAM: {props.total_memory / 1024**3:.1f} GB")
        print(f"  Архитектура: {props.gcnArchName}")
        print(f"  Compute Units: {props.multi_processor_count}")

    print(f"\n✅ Текущая GPU: {torch.cuda.current_device()}")
    print(f"✅ Имя: {torch.cuda.get_device_name()}")
else:
    print("❌ CUDA недоступна!")

print("="*70)

🔍 ПРОВЕРКА GPU
PyTorch версия: 2.5.1+rocm6.2
ROCm версия: 6.2.41133-dd7f95766
Доступных GPU: 1

GPU 0:
  Имя: AMD Radeon RX 7800 XT
  VRAM: 16.0 GB
  Архитектура: gfx1100
  Compute Units: 30

✅ Текущая GPU: 0
✅ Имя: AMD Radeon RX 7800 XT


In [15]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("intfloat/multilingual-e5-base")

sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium."
]
embeddings = model.encode(sentences)

similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)
# [3, 3]

torch.Size([3, 3])


## Remote Inference via Inference Providers
Ensure you have a valid **HF_TOKEN** set in your environment. You can get your token from [your settings page](https://huggingface.co/settings/tokens). Note: running this may incur charges above the free tier.
The following Python example shows how to run the model remotely on HF Inference Providers, automatically selecting an available inference provider for you.
For more information on how to use the Inference Providers, please refer to our [documentation and guides](https://huggingface.co/docs/inference-providers/en/index).

In [19]:
from dotenv import load_dotenv
import os

# Загружаем .env файл
load_dotenv()
hf_key = os.getenv("HF_KEY", "EMPTY")

In [21]:
import os
from huggingface_hub import InferenceClient

client = InferenceClient(
    provider="auto",
    api_key=hf_key,
)

result = client.sentence_similarity(
    "That is a happy person",  # Первый аргумент: исходное предложение
    other_sentences=[          # Второй аргумент: список для сравнения
        "That is a happy dog",
        "That is a very happy person",
        "Today is a sunny day"
    ],
    model="intfloat/multilingual-e5-base"
)

print(result)
# Вывод: [0.xx, 0.yy, 0.zz] - оценки схожести для каждого предложения


[0.9053035974502563, 0.9653543829917908, 0.8199769854545593]
